In [ ]:
! pip install langchain sentence-transformers chromadb

In [ ]:
!pip install -U langchain-community


In [ ]:
! pip install -U langchain-huggingface


In [ ]:
!pip install --upgrade huggingface-hub

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain.vectorstores import Chroma

from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

In [ ]:
import os
from getpass import getpass

HF_token = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"]=HF_token

··········


In [ ]:
# os.environ["RAG_assesment"]=HF_token

In [ ]:
URL = "https://api.freshservice.com/#ticket_attributes"

In [ ]:
data = WebBaseLoader(URL)

In [ ]:
content = data.load()

In [ ]:
print(content)

[Document(metadata={'source': 'https://api.freshservice.com/#ticket_attributes', 'title': 'Service Desk API for Developers | Freshservice', 'description': 'If you are developer who wants to build applications on top of our service desk platform, the Freshservice APIs is all you need.', 'language': 'en-us'}, page_content='        Service Desk API for Developers | Freshservice                             Freshservice    Freshservice SDK Developer Portal Partner Program        Filter by product   All products Freshservice Freshservice for MSPs      Introduction      Introduction     Key differences in terminology     API variations     What\'s New?     Rate limit     Policies     Changelog     Authentication     Schema     Embedding     Attachments     Errors     Pagination     Best Practices     API Index       Tickets      Tickets     Create a Ticket     View a Ticket     Filter Tickets     View List of Tickets     Update a Ticket      Move a Ticket     Delete a Ticket     Delete a Tick

## Chunking - Text Splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [ ]:
chunking=text_splitter.split_documents(content)

In [ ]:
len(chunking)

1955

In [ ]:
chunking[3]

Document(metadata={'source': 'https://api.freshservice.com/#ticket_attributes', 'title': 'Service Desk API for Developers | Freshservice', 'description': 'If you are developer who wants to build applications on top of our service desk platform, the Freshservice APIs is all you need.', 'language': 'en-us'}, page_content='a Time Entry     Tasks     Create a Task     View a Task     View all Tasks     Update a Task     Delete a Task       Changes      Changes     Create a Change     View a Change     View List of Changes     Update a Change     List all Change Fields     Move a Change     Delete a Change     Approvals for Changes     Create an Approval Group for Changes     Update a Change Approval Group     Update approval chain rule for a change     Cancel a Change Approval Group     List all Approval Groups within a Change     View a Change Approval     Send Reminder for a Change Approval     List all Change Approvals     Cancel a Change Approval     Notes     Create a Note     View a 

## Embedding model

In [ ]:
embeddings = HuggingFaceEmbeddings(
     # api_key=HF_token,
    # model_name="thenlper/gte-small"
    model_name="BAAI/bge-base-en-v1.5"
)


/tmp/ipython-input-2526155362.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#!pip install --upgrade huggingface-hub

In [ ]:
vectorstore = Chroma.from_documents(chunking,embeddings)


In [ ]:
llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-alpha",
    model_kwargs={"temperature":0.1,
                  "max_new_tokens":1024,
                  "reptition_penalty":1.1,
                  "return_full_text":False}

)

/tmp/ipython-input-1117359522.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm=HuggingFaceHub(


In [ ]:
retriever= vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":1})

In [ ]:
query ="Give me the curl command to create a ticket"

In [ ]:
docs_rel = retriever.get_relevant_documents(query)
docs_rel

/tmp/ipython-input-3937658708.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_rel = retriever.get_relevant_documents(query)


[Document(metadata={'description': 'If you are developer who wants to build applications on top of our service desk platform, the Freshservice APIs is all you need.', 'title': 'Service Desk API for Developers | Freshservice', 'source': 'https://api.freshservice.com/#ticket_attributes', 'language': 'en-us'}, page_content='puts "response: #{response.code} \\n #{response.body}" EXPAND ↓     Create a Ticket With Custom Fields   Sample code | Curl    1curl -v -u api_key:X -H "Content-Type: application/json" -d \'{ "description": "Details about the issue...", "subject": "Support Needed...", "email": "tom@outerspace.com", "priority": 1, "status": 2, "cc_emails": ["ram@freshservice.com","diana@freshservice.com"], "custom_fields" : { "custom_text" : "This is a custom text box" }, "workspace_id": 3 }\' -X POST \'https://domain.freshservice.com/api/v2/tickets\' EXPAND ↓      Response    1234567891011121314151617181920212223242526272829303132333435363738394041424344454647{\n  "ticket": {\n    "cc_

## Augment Step

In [ ]:
prompt = f"""
<|system|>
You are an AI assistant that follows instructions extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

## Generation step

In [ ]:
qa=RetrievalQA.from_chain_type(llm=llm,retriever=retriever,chain_type="stuff")

In [1]:
response = qa.invoke(prompt)

In [ ]:
print(response["result"])